In [34]:

from pyedhrec import EDHRec
class MyEDHREc(EDHRec):
    def __init__(self):
        super().__init__()
        self._json_page_cards_url = "https://json.edhrec.com/pages/cards/"

    def get_similar_cards(self, cards_name: str):
        formatted_card_name = self.format_card_name(cards_name)
        url = f"{self._json_page_cards_url}{formatted_card_name}.json"
        data = self._get(url)
        return data["similar"]
myedhred = MyEDHREc()
similar_cards = myedhred.get_similar_cards("Pongify")
[x["name"] for x in similar_cards]

['Rapid Hybridization',
 'Beast Within',
 'Reality Shift',
 'Turn to Frog',
 'Chaos Warp',
 'Swan Song']